In [2]:
import sys
# sys.path.append('/Users/bernardoloureiro/template-lib')

from utils.notebookhelpers.helpers import Helpers
from utils.dtos.templateOutputCollection import TemplateOutputCollection
from utils.dtos.variable import Metadata
from utils.dtos.templateOutput import TemplateOutput
from utils.dtos.templateOutput import OutputType
from utils.dtos.templateOutput import ChartType
import logging
import os
from dateutil import parser
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)
!pip install scikit-learn
# !pip install shapely
# !pip install shap-hypetune
# !pip install xgboost
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor,GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix,precision_score
from sklearn.preprocessing import StandardScaler
# import shap
# from shaphypetune import BoostSearch, BoostBoruta, BoostRFE, BoostRFA
from sklearn.metrics import roc_auc_score
# from scipy import stats
# from xgboost import XGBClassifier
# from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import pickle
from sklearn.preprocessing import StandardScaler

In [3]:
#Getting the context
contextId = 'Airline_cargo'
context = Helpers.getOrCreateContext(contextId=contextId, localVars=locals())

inputDatasetParameter=Helpers.get_or_create_input_dataset(
    name="inputDataset3",
    metadata=Metadata(input_name='cleanedd data', is_required=True, tooltip='output transformation'),
    local_context=locals()
)



# inputDatasetParameter = Helpers.get_or_create_input_dataset(
#     name="inputDataset2",
#     metadata=Metadata(input_name='Input Dataset', is_required=True, tooltip='Dataset to apply the transformation'),
#     local_context=locals())



Passengers_In_Parameter = Helpers.get_or_create_input_var(
    name="Passengers_In",
    metadata=Metadata(input_name="Passengers_In", is_required=True, tooltip="All Passengers_In", multiple=False, datatypes=['STRING'],options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)


Freight_In_tonnes_Parameter = Helpers.get_or_create_input_var(
    name="Freight_In_tonnes",
    metadata=Metadata(input_name="Freight_In_tonnes", is_required=True, tooltip="Freight_In_(tonnes)", multiple=False, options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)


Mail_In_tonnes_Parameter = Helpers.get_or_create_input_var(
    name="Mail_In_tonnes",
    metadata=Metadata(input_name="Mail_In_tonnes", is_required=True, tooltip="All Departures_On_Time", multiple=False, options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)


Passengers_Out_Parameter = Helpers.get_or_create_input_var(
    name="Passengers_Out",
    metadata=Metadata(input_name="Passengers_Out", is_required=True, tooltip="All Departures_On_Time", multiple=False, options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)



Freight_Out_tonnes_Parameter = Helpers.get_or_create_input_var(
    name="Freight_Out_tonnes",
    metadata=Metadata(input_name="Freight_Out_tonnes", is_required=True, tooltip="All Departures_On_Time", multiple=False, options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)

Mail_Out_tonnes_Parameter = Helpers.get_or_create_input_var(
    name="Mail_Out_tonnes",
    metadata=Metadata(input_name="Mail_Out_tonnes", is_required=True, tooltip="All Departures_On_Time", multiple=False, options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)

Passengers_Total_Parameter = Helpers.get_or_create_input_var(
    name="Passengers_Total",
    metadata=Metadata(input_name="Passengers_Total", is_required=True, tooltip="All Departures_On_Time", multiple=False, options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)



Freight_Total_tonnes_Parameter = Helpers.get_or_create_input_var(
    name="Freight_Total_tonnes",
    metadata=Metadata(input_name="Freight_Total_tonnes", is_required=True, tooltip="All Departures_On_Time", multiple=False, options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)



Mail_Total_tonnes_Parameter = Helpers.get_or_create_input_var(
    name="Mail_Total_tonnes",
    metadata=Metadata(input_name="Mail_Total_tonnes", is_required=True, tooltip="All Departures_On_Time", multiple=False, options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)




Btwness_Parameter = Helpers.get_or_create_input_var(
    name="Btwness",
    metadata=Metadata(input_name="Btwness", is_required=True, tooltip="Output column", multiple=False,
),
    local_context=locals()
)


closeness_Parameter = Helpers.get_or_create_input_var(
    name="closeness",
    metadata=Metadata(input_name="closeness", is_required=True, tooltip="Output column", multiple=False, 
),
    local_context=locals()
) 


Deg_centrality_Parameter = Helpers.get_or_create_input_var(
    name="Deg_centrality",
    metadata=Metadata(input_name="Deg_centrality", is_required=True, tooltip="Output column", multiple=False, 
),
    local_context=locals()
) 

Freight_In_Out_prct_Parameter = Helpers.get_or_create_input_var(
    name="Freight_In_Out_prct",
    metadata=Metadata(input_name="Freight_In_Out_prct", is_required=True, tooltip="Output column", multiple=False, 
),
    local_context=locals()
) 



Passengers_In_Out_prct_Parameter = Helpers.get_or_create_input_var(
    name="Passengers_In_Out_prct",
    metadata=Metadata(input_name="Passengers_In_Out_prct", is_required=True, tooltip="Output column", multiple=False, 
),
    local_context=locals()
) 


Mail_In_Out_prct_Parameter = Helpers.get_or_create_input_var(
    name="Mail_In_Out_prct",
    metadata=Metadata(input_name="Mail_In_Out_prct", is_required=True, tooltip="Output column", multiple=False, 
),
    local_context=locals()
) 



W_Passengers_In_Parameter = Helpers.get_or_create_input_var(
    name="W_Passengers_In",
    metadata=Metadata(input_name="W_Passengers_In", is_required=True, tooltip="Output column", multiple=False, 
),
    local_context=locals()
) 


W_Passengers_Out_Parameter = Helpers.get_or_create_input_var(
    name="W_Passengers_Out",
    metadata=Metadata(input_name="W_Passengers_Out", is_required=True, tooltip="Output column", multiple=False, 
),
    local_context=locals()
) 


W_Passengers_Total_Parameter = Helpers.get_or_create_input_var(
    name="W_Passengers_Total",
    metadata=Metadata(input_name="W_Passengers_Total", is_required=True, tooltip="Output column", multiple=False, 
),
    local_context=locals()
) 


flights_total_Parameter = Helpers.get_or_create_input_var(
    name="flights_total",
    metadata=Metadata(input_name="flights_total", is_required=True, tooltip="Output column", multiple=False, 
),
    local_context=locals()
) 

Total_wt_Parameter = Helpers.get_or_create_input_var(
    name="Total_wt",
    metadata=Metadata(input_name="Total_wt", is_required=True, tooltip="Output column", multiple=False, 
),
    local_context=locals()
) 

Avg_flight_wt_Parameter = Helpers.get_or_create_input_var(
    name="Avg_flight_wt",
    metadata=Metadata(input_name="Avg_flight_wt", is_required=True, tooltip="Output column", multiple=False, 
),
    local_context=locals()
) 












outputDatasetParameter=Helpers.get_or_create_output_dataset(
 name="outputDataset3",
    metadata=Metadata(input_name='Xtrain', is_required=True, tooltip='Dataset name to be created after the transformation'),
    local_context=locals()
)


#Model Save

model_to_saveParameter = Helpers.get_or_create_input_var(
    name="model_to_save",
    metadata=Metadata(input_name="RF cargo Model", is_required=False, tooltip="If you want your model to be saved and be used on for prediction, it is required.", multiple=False, datatypes=['STRING'], options=['CONSTANT']),
    local_context=locals()
)

In [4]:
#Getting the parameters
airline=inputDatasetParameter.value



Passengers_In=    Passengers_In_Parameter.value
Freight_In_tonnes=    Freight_In_tonnes_Parameter.value
Mail_In_tonnes = Mail_In_tonnes_Parameter.value
Passengers_Out = Passengers_Out_Parameter.value


Freight_Out_tonnes = Freight_Out_tonnes_Parameter.value
Mail_Out_tonnes=Mail_Out_tonnes_Parameter.value
Freight_Total_tonnes =  Freight_Total_tonnes_Parameter.value

Passengers_Total = Passengers_Total_Parameter.value
Mail_Total_tonnes = Mail_Total_tonnes_Parameter.value

Btwness = Btwness_Parameter.value
closeness = closeness_Parameter.value
Deg_centrality = Deg_centrality_Parameter.value

Freight_In_Out_prct =     Freight_In_Out_prct_Parameter.value
Passengers_In_Out_prct=    Passengers_In_Out_prct_Parameter.value
Mail_In_Out_prct =    Mail_In_Out_prct_Parameter.value
W_Passengers_In=    W_Passengers_In_Parameter.value
W_Passengers_Out =    W_Passengers_Out_Parameter.value
W_Passengers_Total =    W_Passengers_Total_Parameter.value
flights_total=    flights_total_Parameter.value
Total_wt =    Total_wt_Parameter.value
Avg_flight_wt =    Avg_flight_wt_Parameter.value




outputDataset3=outputDatasetParameter.value

model_to_save=model_to_saveParameter.value


In [5]:
airline =  Helpers.getEntityData(context, airline)

In [6]:
airline.head(2)

,Deg_centrality,closeness,Btwness,Passengers_In,Freight_In_tonnes,Mail_In_tonnes,Passengers_Out,Freight_Out_tonnes,Mail_Out_tonnes,Passengers_Total,...,Freight_Total_tonnes,Freight_In_Out_prct,Passengers_In_Out_prct,Mail_In_Out_prct,W_Passengers_In,W_Passengers_Out,W_Passengers_Total,flights_total,Total_wt,Avg_flight_wt
1,0.325359,0.44186,0.02568,6.0,0.0,0.0,16.0,18.7,0.0,22.0,...,18.7,0.0,-2.444444,0.0,0.408,1.088,1.496,0.048889,0.324436,6.625618
2,0.325359,0.44186,0.02568,13.0,0.0,0.0,7.0,23.9,0.0,20.0,...,23.9,0.0,-1.466667,0.0,0.884,0.476,1.360,0.044444,0.347858,8.305657


In [7]:
airline_loged = airline.apply(np.log1p)
airline_loged.replace((np.inf, -np.inf, np.nan), 0,inplace=True)

In [10]:
airline_loged.head(2)

,Deg_centrality,closeness,Btwness,Passengers_In,Freight_In_tonnes,Mail_In_tonnes,Passengers_Out,Freight_Out_tonnes,Mail_Out_tonnes,Passengers_Total,...,Freight_Total_tonnes,Freight_In_Out_prct,Passengers_In_Out_prct,Mail_In_Out_prct,W_Passengers_In,W_Passengers_Out,W_Passengers_Total,flights_total,Total_wt,Avg_flight_wt
1,0.281683,0.365934,0.025356,1.945910,0.0,0.0,2.833213,2.980619,0.0,3.135494,...,2.980619,0.0,0.0,0.0,0.342170,0.736207,0.914689,0.047731,0.280986,2.031513
2,0.281683,0.365934,0.025356,2.639057,0.0,0.0,2.079442,3.214868,0.0,3.044522,...,3.214868,0.0,0.0,0.0,0.633397,0.389336,0.858662,0.043485,0.298517,2.230623


In [11]:
Xtrain = airline_loged.drop(['Freight_Total_tonnes'],axis=1)

Ytrain=airline_loged['Freight_Total_tonnes']

In [13]:
len(Xtrain),len(Ytrain)

(4492, 4492)

In [15]:
X_train,X_test,Y_train,Y_test = train_test_split(Xtrain,Ytrain,test_size=0.2,random_state=1200)
print(X_train.shape , X_test.shape)

(3593, 20) (899, 20)


In [16]:
sc = StandardScaler()
X_train_norm = sc.fit_transform(X_train)
X_test_norm = sc.transform(X_test)

# Model Training

In [ ]:

# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 150],
    'max_features': [5,8,16,13 ],
    'min_samples_leaf': [3, 5, 7],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 800, 1500, 1000]
}


# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90],
    'max_features': [5,8 ],
    'min_samples_leaf': [3, 4],
    'min_samples_split': [8, 3],
    'n_estimators': [100,50]
}

In [18]:
rf = RandomForestRegressor(random_state=106)

In [19]:
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 2, n_jobs = -1, verbose = 2)

In [21]:
# Fit the grid search to the data
grid_search.fit(X_train_norm, Y_train)

Fitting 2 folds for each of 32 candidates, totalling 64 fits


GridSearchCV(cv=2, estimator=RandomForestRegressor(random_state=106), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [80, 90],
                         'max_features': [5, 8], 'min_samples_leaf': [3, 4],
                         'min_samples_split': [8, 3],
                         'n_estimators': [100, 50]},
             verbose=2)

In [24]:
len(Y_test)

899

In [23]:
Y_test.replace((np.inf, -np.inf, np.nan), 0,inplace=True)

In [26]:
y_pred = grid_search.predict(X_test_norm)

In [28]:
mae = metrics.mean_absolute_error(Y_test, y_pred)
mse = metrics.mean_squared_error(Y_test, y_pred)
rmse = np.sqrt(mse) # or mse**(0.5)  
r2 = metrics.r2_score(Y_test,y_pred)

In [31]:
result = pd.DataFrame.from_dict({'MAE':[mae],'MSE':[mse],'RMSE':[rmse],'R2squres':[r2]})

In [32]:
result

,MAE,MSE,RMSE,R2squres
0,0.055659,0.009667,0.098323,0.998231


In [ ]:
if model_to_save is None or len(model_to_save) == 0 :
    { print('No model to be saved')}
else:
    import pickle
    artifactsDir = Helpers.getOrCreateArtifactsDir(context, artifactsId = model_to_save)
    with open(artifactsDir + '/RF_cargo.pkl', 'wb') as handle:
        pickle.dump(grid_search, handle)

filename = 'XGB_model_shap.sav'
pickle.dump(model, open(filename, 'wb'))

result= pd.DataFrame(model.predict(X_test_norm))

In [ ]:
outputCollection = Helpers.createOutputCollection(context)
out = Helpers.createTemplateOutputDataset(context=context, outputName=outputDataset3, dataFrame=result)

# out = Helpers.createTemplateOutputDataset(context=context, outputName=outputDataset3, dataFrame=Xtrain)
# out2 = Helpers.createTemplateOutputDataset(context=context, outputName=outputDataset4, dataFrame=X_test_norm)
# out3 = Helpers.createTemplateOutputDataset(context=context, outputName=outputDataset5, dataFrame=Y_train)
# out4 = Helpers.createTemplateOutputDataset(context=context, outputName=outputDataset6, dataFrame=Y_test)
outputCollection.addTemplateOutput(out)
# outputCollection.addTemplateOutput(out2)
# outputCollection.addTemplateOutput(out3)
# outputCollection.addTemplateOutput(out4)
Helpers.save(context)